## Diseño del modelo de simulación

En esta evaluación se implementa una simulación simplificada de selección natural inspirada en el video “Simulating Natural Selection”. El objetivo es modelar criaturas (blobs) que se mueven en un mapa rectangular, buscan comida, consumen energía, pueden morir y, opcionalmente, desarrollar una nueva característica (por ejemplo, mayor velocidad).

### Mapa

- El entorno es un mapa 2D rectangular de tamaño fijo.
- Cada posición se representa con coordenadas (x, y) en punto flotante.
- Los blobs y la comida se mantienen siempre dentro de los límites del mapa.

### Clase `Food`

La clase `Food` representa una unidad de comida en el mapa:

- Atributos principales:
  - `x`, `y`: posición en el mapa.
  - `energy_value`: cantidad de energía que entrega al blob que la consume.
  - `consumed`: indica si ya fue consumida.
- La comida no tiene comportamiento propio; simplemente está disponible para que los blobs la encuentren.

### Clase `Blob`

La clase `Blob` modela a cada criatura de la simulación:

- Atributos principales:
  - `id`: identificador único.
  - `x`, `y`: posición actual.
  - `home_x`, `home_y`: punto de origen (casa).
  - `speed`: velocidad de movimiento.
  - `energy`: energía actual.
  - `base_energy_loss`: energía que pierde en cada paso.
  - `has_speed_mutation`: indica si desarrolló la mutación de velocidad.
  - `alive`: indica si el blob sigue vivo.
- Comportamientos clave:
  - Cálculo de distancia a un objetivo.
  - Movimiento hacia la comida más cercana.
  - Movimiento aleatorio cuando no hay comida disponible.
  - Consumo de energía en cada paso.
  - Verificación de muerte cuando la energía llega a cero o menos.
  - Consumo de comida cuando se encuentra suficientemente cerca.
  - Reinicio de posición al inicio de un nuevo día.

En esta primera parte se implementan únicamente estas dos clases y una prueba simple para verificar su funcionamiento básico antes de integrar el agente gestor y el resto de la lógica de selección natural.


In [7]:
import random
import math
from typing import List

MAP_WIDTH = 100
MAP_HEIGHT = 100

STEPS_PER_DAY = 200
INITIAL_BLOB_COUNT = 20
INITIAL_FOOD_COUNT = 40

INITIAL_ENERGY = 100.0
ENERGY_LOSS_PER_STEP = 0.5
ENERGY_FROM_FOOD = 30.0
REPRODUCTION_THRESHOLD = 120.0
MUTATION_PROBABILITY = 0.2
MUTATION_SPEED_BOOST = 0.5


In [8]:
MAX_BLOBS = 50

In [9]:
import tkinter as tk
from tkinter import ttk

In [10]:
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour
import asyncio

In [22]:
async def _no_connect(self, *args, **kwargs):
    return

Agent._async_connect = _no_connect


## Implementación de la clase `Food`

En esta sección se implementa la clase `Food`, que representa unidades de comida distribuidas en el mapa. Cada objeto `Food` almacena su posición, la cantidad de energía que aporta a los blobs y un indicador de si ya fue consumido. Esta clase se utilizará posteriormente dentro de la lógica de los blobs y del agente gestor de la simulación.


In [12]:
class Food:
    def __init__(self, x: float, y: float, energy_value: float = ENERGY_FROM_FOOD):
        self.x = x
        self.y = y
        self.energy_value = energy_value
        self.consumed = False

    @property
    def position(self):
        return (self.x, self.y)


## Implementación de la clase `Blob`

En esta sección se implementa la clase `Blob`, que modela a las criaturas de la simulación. Cada blob tiene una posición en el mapa, una cantidad de energía y una velocidad de movimiento. En cada paso de la simulación, el blob:

1. Busca la comida más cercana que no haya sido consumida.
2. Se mueve hacia ella si existe, o realiza un movimiento aleatorio si no encuentra comida.
3. Pierde una cantidad fija de energía por moverse.
4. Puede consumir comida si se encuentra a una distancia suficientemente pequeña.

Si la energía del blob llega a cero o menos, se considera que ha muerto. Además, se incluye un método para reiniciar su posición al inicio de un nuevo día de simulación.


In [13]:
class Blob:
    _next_id = 0

    def __init__(
        self,
        x: float,
        y: float,
        speed: float = 1.0,
        energy: float = INITIAL_ENERGY,
        has_speed_mutation: bool = False
    ):
        self.id = Blob._next_id
        Blob._next_id += 1
        self.x = x
        self.y = y
        self.home_x = x
        self.home_y = y
        self.speed = speed
        self.energy = energy
        self.base_energy_loss = ENERGY_LOSS_PER_STEP
        self.has_speed_mutation = has_speed_mutation
        self.alive = True

    @property
    def position(self):
        return (self.x, self.y)

    def distance_to(self, target_x: float, target_y: float) -> float:
        return math.sqrt((self.x - target_x) ** 2 + (self.y - target_y) ** 2)

    def _move_towards(self, target_x: float, target_y: float):
        dx = target_x - self.x
        dy = target_y - self.y
        dist = math.sqrt(dx * dx + dy * dy)
        if dist == 0:
            return
        ux = dx / dist
        uy = dy / dist
        self.x += ux * self.speed
        self.y += uy * self.speed
        self.x = min(max(self.x, 0), MAP_WIDTH)
        self.y = min(max(self.y, 0), MAP_HEIGHT)

    def _random_move(self):
        angle = random.uniform(0, 2 * math.pi)
        dx = math.cos(angle) * self.speed
        dy = math.sin(angle) * self.speed
        self.x += dx
        self.y += dy
        self.x = min(max(self.x, 0), MAP_WIDTH)
        self.y = min(max(self.y, 0), MAP_HEIGHT)

    def step(self, food_list: List[Food]):
        if not self.alive:
            return
        visible_food = [f for f in food_list if not f.consumed]
        target_food = None
        min_dist = float("inf")
        for food in visible_food:
            d = self.distance_to(food.x, food.y)
            if d < min_dist:
                min_dist = d
                target_food = food
        if target_food is not None:
            self._move_towards(target_food.x, target_food.y)
        else:
            self._random_move()
        self.energy -= self.base_energy_loss
        if self.energy <= 0:
            self.alive = False
            return
        for food in visible_food:
            if not food.consumed and self.distance_to(food.x, food.y) < 1.0:
                food.consumed = True
                self.energy += food.energy_value
                break

    def reset_for_new_day(self):
        self.x = self.home_x
        self.y = self.home_y

    def is_dead(self) -> bool:
        return not self.alive


## Prueba rápida de `Blob` y `Food`

Antes de construir el agente gestor y la lógica completa de selección natural, se realiza una prueba simple de las clases `Blob` y `Food`. En esta prueba se:

- Generan varias unidades de comida en posiciones aleatorias del mapa.
- Se crea un blob inicial en el centro del mapa.
- Se ejecutan varios pasos de simulación, observando la posición, energía y estado (vivo o muerto) del blob.

Esto permite validar que las clases básicas funcionan correctamente y que no hay errores de sintaxis o de lógica en la actualización de estado de los blobs.


In [14]:
food_items = [
    Food(random.uniform(0, MAP_WIDTH), random.uniform(0, MAP_HEIGHT))
    for _ in range(5)
]

blob = Blob(x=50, y=50, speed=1.5)

for step in range(20):
    blob.step(food_items)
    print(f"Step {step:02d} | Pos: ({blob.x:.2f}, {blob.y:.2f}) | Energy: {blob.energy:.2f} | Alive: {blob.alive}")


Step 00 | Pos: (49.52, 51.42) | Energy: 99.50 | Alive: True
Step 01 | Pos: (49.03, 52.84) | Energy: 99.00 | Alive: True
Step 02 | Pos: (48.55, 54.26) | Energy: 98.50 | Alive: True
Step 03 | Pos: (48.07, 55.68) | Energy: 98.00 | Alive: True
Step 04 | Pos: (47.59, 57.10) | Energy: 127.50 | Alive: True
Step 05 | Pos: (47.60, 55.60) | Energy: 127.00 | Alive: True
Step 06 | Pos: (47.61, 54.10) | Energy: 126.50 | Alive: True
Step 07 | Pos: (47.62, 52.60) | Energy: 126.00 | Alive: True
Step 08 | Pos: (47.62, 51.10) | Energy: 125.50 | Alive: True
Step 09 | Pos: (47.63, 49.60) | Energy: 125.00 | Alive: True
Step 10 | Pos: (47.64, 48.10) | Energy: 124.50 | Alive: True
Step 11 | Pos: (47.65, 46.60) | Energy: 124.00 | Alive: True
Step 12 | Pos: (47.66, 45.10) | Energy: 123.50 | Alive: True
Step 13 | Pos: (47.67, 43.60) | Energy: 123.00 | Alive: True
Step 14 | Pos: (47.68, 42.10) | Energy: 122.50 | Alive: True
Step 15 | Pos: (47.69, 40.60) | Energy: 152.00 | Alive: True
Step 16 | Pos: (49.19, 40.72

## Agente gestor de blobs (`SimulationManager`)

Además de las clases `Blob` y `Food`, la simulación requiere un agente central que controle la dinámica global. En este caso se implementa la clase `SimulationManager`, que cumple el rol de “host” o gestor de blobs:

### Responsabilidades principales

- Mantener la lista de blobs y de comida activos en el entorno.
- Definir las dimensiones del mapa y el número de pasos por día.
- Crear los blobs iniciales en posiciones aleatorias a lo largo de los bordes del mapa.
- Generar comida nueva al inicio de cada día.
- Actualizar el estado de cada blob en cada paso de simulación.
- Aplicar una regla simple de selección natural y reproducción al final de cada día.
- Controlar un tamaño máximo de población de blobs.

### Ciclo de simulación por día

Para cada día de simulación, el `SimulationManager` realiza las siguientes etapas:

1. Incrementa el contador de día.
2. Reinicia la posición de los blobs vivos a su punto de origen.
3. Genera una cantidad fija de comida distribuida aleatoriamente en el mapa.
4. Ejecuta un número fijo de pasos; en cada paso, cada blob vivo actualiza su estado usando el método `step`.
5. Al finalizar el día, aplica selección natural y reproducción:
   - Los blobs muertos o con muy poca energía se eliminan.
   - Los blobs con energía suficiente se consideran más aptos.
   - Algunos blobs “aptos” pueden generar descendencia, respetando un número máximo de blobs.
   - Durante la creación de descendencia se puede producir una mutación de velocidad, modelando la aparición de una nueva característica (por ejemplo, blobs más rápidos).

El resultado es una población que puede cambiar en tamaño y en distribución de velocidades a lo largo de los días, emulando el efecto de selección natural de forma simplificada.


In [15]:
class SimulationManager:
    def __init__(
        self,
        max_blobs: int = MAX_BLOBS,
        initial_blob_count: int = INITIAL_BLOB_COUNT,
        daily_food_count: int = INITIAL_FOOD_COUNT
    ):
        self.max_blobs = max_blobs
        self.daily_food_count = daily_food_count
        self.day = 0
        self.blobs: List[Blob] = []
        self.food: List[Food] = []
        self._create_initial_blobs(initial_blob_count)

    def _random_edge_position(self):
        side = random.choice(["top", "bottom", "left", "right"])
        if side in ("top", "bottom"):
            x = random.uniform(0, MAP_WIDTH)
            y = 0 if side == "top" else MAP_HEIGHT
        else:
            y = random.uniform(0, MAP_HEIGHT)
            x = 0 if side == "left" else MAP_WIDTH
        return x, y

    def _create_initial_blobs(self, count: int):
        self.blobs = []
        for _ in range(count):
            x, y = self._random_edge_position()
            blob = Blob(x, y, speed=1.0)
            self.blobs.append(blob)

    def _spawn_food(self):
        self.food = []
        for _ in range(self.daily_food_count):
            x = random.uniform(0, MAP_WIDTH)
            y = random.uniform(0, MAP_HEIGHT)
            self.food.append(Food(x, y))

    def _reset_blobs_for_new_day(self):
        for blob in self.blobs:
            if blob.alive:
                blob.reset_for_new_day()

    def _apply_selection_and_reproduction(self):
        survivors = [b for b in self.blobs if b.alive and b.energy > 0]
        new_generation: List[Blob] = []
        for blob in survivors:
            new_generation.append(blob)
            if blob.energy >= REPRODUCTION_THRESHOLD and len(new_generation) < self.max_blobs:
                child_speed = blob.speed
                child_has_mutation = blob.has_speed_mutation
                if random.random() < MUTATION_PROBABILITY:
                    child_speed = blob.speed + MUTATION_SPEED_BOOST
                    child_has_mutation = True
                child = Blob(
                    blob.home_x,
                    blob.home_y,
                    speed=child_speed,
                    energy=INITIAL_ENERGY,
                    has_speed_mutation=child_has_mutation
                )
                new_generation.append(child)
                if len(new_generation) >= self.max_blobs:
                    break
        self.blobs = new_generation

    def simulate_day(self):
        self.day += 1
        self._reset_blobs_for_new_day()
        self._spawn_food()
        for _ in range(STEPS_PER_DAY):
            for blob in self.blobs:
                if blob.alive:
                    blob.step(self.food)
        self._apply_selection_and_reproduction()

    def population_stats(self):
        total = len(self.blobs)
        alive = sum(1 for b in self.blobs if b.alive)
        mutated = sum(1 for b in self.blobs if b.has_speed_mutation)
        avg_speed = sum(b.speed for b in self.blobs) / total if total > 0 else 0.0
        return {
            "total": total,
            "alive": alive,
            "mutated": mutated,
            "avg_speed": avg_speed,
        }

    def run(self, days: int):
        history = []
        for _ in range(days):
            self.simulate_day()
            stats = self.population_stats()
            history.append((self.day, stats))
        return history


## Simulación sin GUI: prueba del agente gestor

Antes de construir la interfaz gráfica, se realiza una simulación en modo texto utilizando el `SimulationManager`. En esta prueba:

- Se crea un gestor con un número inicial de blobs y un límite máximo de población.
- Se ejecuta la simulación durante varios días.
- Al final de cada día se registran estadísticas básicas:
  - tamaño total de la población,
  - cantidad de blobs vivos,
  - cantidad de blobs con mutación de velocidad,
  - velocidad promedio de la población.

Estos resultados permiten verificar que:
- la población evoluciona a lo largo del tiempo,
- la selección natural elimina blobs con baja energía,
- aparecen gradualmente blobs con la nueva característica (mayor velocidad) debido a la mutación.


In [16]:
manager = SimulationManager(
    max_blobs=MAX_BLOBS,
    initial_blob_count=INITIAL_BLOB_COUNT,
    daily_food_count=INITIAL_FOOD_COUNT
)

history = manager.run(days=10)

for day, stats in history:
    print(
        f"Day {day:02d} | "
        f"Total: {stats['total']:2d} | "
        f"Alive: {stats['alive']:2d} | "
        f"Mutated: {stats['mutated']:2d} | "
        f"Avg speed: {stats['avg_speed']:.2f}"
    )


Day 01 | Total: 19 | Alive: 19 | Mutated:  0 | Avg speed: 1.00
Day 02 | Total: 14 | Alive: 14 | Mutated:  0 | Avg speed: 1.00
Day 03 | Total: 14 | Alive: 14 | Mutated:  0 | Avg speed: 1.00
Day 04 | Total: 16 | Alive: 16 | Mutated:  1 | Avg speed: 1.03
Day 05 | Total: 17 | Alive: 17 | Mutated:  2 | Avg speed: 1.06
Day 06 | Total: 18 | Alive: 18 | Mutated:  5 | Avg speed: 1.14
Day 07 | Total: 18 | Alive: 18 | Mutated: 10 | Avg speed: 1.28
Day 08 | Total: 20 | Alive: 20 | Mutated: 14 | Avg speed: 1.40
Day 09 | Total: 22 | Alive: 22 | Mutated: 17 | Avg speed: 1.48
Day 10 | Total: 22 | Alive: 22 | Mutated: 19 | Avg speed: 1.55


## Diseño de la interfaz gráfica (GUI)

Para visualizar la simulación en tiempo real se implementa una interfaz gráfica utilizando `tkinter`. La GUI permite observar la posición de los blobs y de la comida en un mapa 2D, además de controlar la velocidad de actualización de la simulación.

### Objetivos de la GUI

- Mostrar en tiempo real la posición de los blobs y de la comida.
- Distinguir a los blobs que han desarrollado la nueva característica (mayor velocidad).
- Permitir ajustar la velocidad de la simulación mediante un control deslizante.
- Poder iniciar y pausar la simulación desde un botón.
- Mantener la lógica de la simulación separada de la lógica de dibujo.

### Integración con el agente gestor

Para no modificar la lógica principal de la simulación, se crea una variante del gestor llamada `SimulationManagerWithGUI`, que hereda de `SimulationManager` y agrega dos métodos:

- `start_new_day_gui()`: prepara un nuevo día de simulación (reinicia blobs, genera comida y reinicia el contador de pasos).
- `step_one_tick_gui()`: ejecuta un paso de simulación. Cuando se completan los pasos de un día, aplica selección natural y reproduce blobs, y luego inicia un nuevo día.

La clase `SimulationGUI` recibe una instancia de `SimulationManagerWithGUI` y se encarga de:

- Dibujar los blobs y la comida sobre un `Canvas`.
- Ejecutar periódicamente pasos de simulación usando `after`, de acuerdo con la velocidad seleccionada.
- Mostrar en una etiqueta información agregada como el día actual, el tamaño de la población y la cantidad de blobs mutados.

De esta forma, la simulación puede ejecutarse en modo texto (solo con el gestor) o en modo gráfico (con la GUI), cumpliendo con el requisito de que la interfaz gráfica no esté activa el 100% del tiempo.


In [17]:
class SimulationManagerWithGUI(SimulationManager):
    def __init__(
        self,
        max_blobs: int = MAX_BLOBS,
        initial_blob_count: int = INITIAL_BLOB_COUNT,
        daily_food_count: int = INITIAL_FOOD_COUNT
    ):
        super().__init__(max_blobs=max_blobs, initial_blob_count=initial_blob_count, daily_food_count=daily_food_count)
        self.current_step = 0

    def start_new_day_gui(self):
        self.day += 1
        self._reset_blobs_for_new_day()
        self._spawn_food()
        self.current_step = 0

    def step_one_tick_gui(self):
        if self.current_step < STEPS_PER_DAY:
            for blob in self.blobs:
                if blob.alive:
                    blob.step(self.food)
            self.current_step += 1
        else:
            self._apply_selection_and_reproduction()
            self.start_new_day_gui()


## Uso de la GUI

Para utilizar la interfaz gráfica:

1. Se crea primero un objeto `SimulationManagerWithGUI` indicando los parámetros de la simulación (número máximo de blobs, cantidad inicial de blobs, cantidad diaria de comida).
2. A partir de este gestor se crea un objeto `SimulationGUI`.
3. Se llama al método `run()` de la GUI para iniciar la ventana.

Dentro de la ventana:

- El botón “Iniciar / Pausar” permite arrancar o detener temporalmente la simulación.
- El control deslizante ajusta el tiempo de espera entre actualizaciones (en milisegundos), lo que equivale a cambiar la velocidad de la simulación.
- Los puntos o círculos representan:
  - comida disponible en el mapa,
  - blobs normales,
  - blobs mutados (por ejemplo, con un color distinto).
- En la parte inferior se muestra un resumen con el día actual, el tamaño de la población total, el número de blobs mutados y la velocidad promedio.

Esta GUI puede iniciarse en cualquier momento mientras se desarrolla la evaluación, sin necesidad de que esté activa durante toda la ejecución del notebook.


In [18]:
class SimulationGUI:
    def __init__(self, manager: SimulationManagerWithGUI):
        self.manager = manager
        self.root = tk.Tk()
        self.root.title("Simulación de selección natural - blobs")
        self.root.configure(bg="#020617")
        
        self.running = False
        self.delay_ms = 80
        self.width = 800
        self.height = 500

        self.canvas = tk.Canvas(self.root, width=self.width, height=self.height, bg="#020617", highlightthickness=0)
        self.canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=(10, 0))

        top_frame = ttk.Frame(self.root)
        top_frame.pack(side=tk.TOP, fill=tk.X, padx=10, pady=5)

        title_label = ttk.Label(top_frame, text="Simulación de selección natural", font=("Segoe UI", 12, "bold"))
        title_label.pack(side=tk.LEFT)

        legend_label = ttk.Label(
            top_frame,
            text="■ Comida (verde)   ● Blob normal (celeste)   ● Blob mutado (naranja)",
            font=("Segoe UI", 9)
        )
        legend_label.pack(side=tk.RIGHT)

        control_frame = ttk.Frame(self.root)
        control_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=10, pady=10)

        self.start_button = ttk.Button(control_frame, text="Iniciar / Pausar", command=self.toggle_running)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.speed_scale = ttk.Scale(control_frame, from_=10, to=300, value=self.delay_ms, command=self.on_speed_change)
        self.speed_scale.pack(side=tk.LEFT, padx=5)

        self.speed_label = ttk.Label(control_frame, text=f"Velocidad (ms): {self.delay_ms}")
        self.speed_label.pack(side=tk.LEFT, padx=5)

        self.info_label = ttk.Label(control_frame, text="", anchor="e")
        self.info_label.pack(side=tk.RIGHT, padx=5)

        style = ttk.Style()
        try:
            style.theme_use("clam")
        except:
            pass

        self.manager.start_new_day_gui()
        self.update_canvas()

    def toggle_running(self):
        self.running = not self.running
        if self.running:
            self.loop()

    def on_speed_change(self, value):
        self.delay_ms = int(float(value))
        self.speed_label.config(text=f"Velocidad (ms): {self.delay_ms}")

    def coord_to_canvas(self, x, y):
        cx = x / MAP_WIDTH * self.width
        cy = y / MAP_HEIGHT * self.height
        return cx, cy

    def update_canvas(self):
        self.canvas.delete("all")
        border_margin = 10
        self.canvas.create_rectangle(
            border_margin,
            border_margin,
            self.width - border_margin,
            self.height - border_margin,
            outline="#1f2937"
        )
        for food in self.manager.food:
            if not food.consumed:
                x, y = self.coord_to_canvas(food.x, food.y)
                r = 3
                self.canvas.create_rectangle(x - r, y - r, x + r, y + r, fill="#4ade80", outline="#16a34a")
        for blob in self.manager.blobs:
            if not blob.alive:
                continue
            x, y = self.coord_to_canvas(blob.x, blob.y)
            r = 5
            color = "#38bdf8"
            outline = "#0ea5e9"
            if blob.has_speed_mutation:
                color = "#fb923c"
                outline = "#f97316"
            self.canvas.create_oval(x - r, y - r, x + r, y + r, fill=color, outline=outline, width=1.3)
        stats = self.manager.population_stats()
        self.info_label.config(
            text=f"Día: {self.manager.day}   |   Población: {stats['total']}   |   Mutados: {stats['mutated']}   |   Vel prom: {stats['avg_speed']:.2f}"
        )

    def loop(self):
        if self.running:
            self.manager.step_one_tick_gui()
            self.update_canvas()
            self.root.after(self.delay_ms, self.loop)

    def run(self):
        self.root.mainloop()


## Uso de SPADE como agente gestor

Para cumplir con el modelo de agentes del curso, se implementa un agente SPADE
llamado `SimulationAgent`. Este agente envuelve al `SimulationManagerWithGUI`
y ejecuta periódicamente la lógica de la simulación mediante un `CyclicBehaviour`.

Los blobs y la comida **no son agentes**, sino objetos
que contienen su propio estado. El `SimulationAgent` es el único agente del sistema
y se encarga de avanzar la simulación paso a paso.

Como la práctica no requiere comunicación entre agentes ni uso de un servidor XMPP,
el agente se ejecuta en **modo local/offline**, permitiendo que la simulación funcione
sin necesidad de servicios externos.


In [19]:
class SimulationAgent(Agent):
    def __init__(self, jid, password, manager: SimulationManagerWithGUI):
        super().__init__(jid, password)
        self.manager = manager

    class StepBehaviour(CyclicBehaviour):
        async def run(self):
            self.agent.manager.step_one_tick_gui()
            await asyncio.sleep(0.05)  

    async def setup(self):
        b = self.StepBehaviour()
        self.add_behaviour(b)


In [20]:
manager_gui = SimulationManagerWithGUI(
    max_blobs=MAX_BLOBS,
    initial_blob_count=INITIAL_BLOB_COUNT,
    daily_food_count=INITIAL_FOOD_COUNT
)
gui = SimulationGUI(manager_gui)
agent = SimulationAgent("test@localhost", "1234", manager_gui)


In [23]:
await agent.start(auto_register=False)   
gui.run()
await agent.stop()